# Basic Usage

In this notebook, we demonstrate how the `comsol_mesh` library can be used to import both geometry and eigenmodes from COMSOL output. We also show how these can be used to compute the relative frequency shifts induced by analyte physisorption in the limit that the analyte mass is much smaller than the device mass.

In [1]:
import numpy as np
import bokeh.palettes as palettes

from pathlib import Path
from comsol_mesh import *

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import Column, gridplot, Row

palette = palettes.Category10[10]

output_notebook()

Loading BokehJS ...

## Importing geometry

Geometry can be exported from the industry standard software package COMSOL in the form of an `.mphtxt` file. Each `.mphtxt` file contains header information and one or more _COMSOL objects_. Using the `COMSOLObjects` class we can parse this information into a Python readable format.

Each instance of `COMSOLObjects` contains a list of _COMSOL objects_ which are Python dictionaries describing the points, edges, triangles and tetrahedra of each object. To peform geometric computation on these entities we can parse this dictionary into an instance of the `Mesh` class.

In [2]:
# Read .mphtxt file
comsol_objs = COMSOLObjects.from_file('data/shear_device.mphtxt')

# Select first COMSOL object
cobj = comsol_objs[0]

mesh = Mesh.from_comsol_obj(cobj)
mesh

Mesh(n_points=2823, n_tetrahedra=13851)

In [3]:
# Plot the points of the mesh
show(plot_points(mesh.points))

## Linear interpolation

Eigenmodes can be imported using the `COMSOLEigenmodes` class. This class stores information the values of the eigenmodes on the points of the mesh but lacks information about the elements of the mesh (i.e. connections between points on the mesh)

Linear functions on the mesh can be defined using the `Field` class. To associate the point-value information about the eigenmodes in a `COMSOLEigenmodes` instance with a mesh we can create a linear function on the mesh using the `Field.from_comsol_field(mesh, comsol_field)` class method. Geometric computation can then be performed on the linear function defined by the `Field` instance.

In [4]:
# 1. Create linear interpolant
f = lambda x: np.cos(x[0] ** 2 + x[1] ** 2) + x[2]  # Arbitrary scalar function in 3-dimensions
f_values = np.apply_along_axis(f, 1, mesh.points).reshape((-1, 1)) #

field = Field(mesh, f_values)

print(f'Integral: {field.integrate()}')
print(f'L2 norm: {field.L2_norm()}')

Integral: [0.28295549]
L2 norm: [0.5351687]


In [5]:
# 2. Load and define eigenmode interpolants
cemodes = COMSOLEigenmodes.from_file('data/shear_device_eigenmodes.csv')
modes_field = Field.from_comsol_field(mesh, cemodes)

modes_field

Field(mesh=Mesh(n_points=2823, n_tetrahedra=13851), field_shape=(10, 3))

## Modal mass

In structural analysis, the _modal-mass_ $m$ of an eigenmode $\phi(x)$ is defined as,
$$ m = \int_V \rho(x) \, \|\phi(x)\|^2 \, d^3x,$$
where $\rho(x)$ is the density of the device and the integration is performed over the volume $V$ of the device. Modal-mass eigenmode normalisation sets the modal-mass of each eigenmode to unity.

For this example, to confirm that each eigenmode is normalised to unity we use the `L2_norm` method of the `Field` class and specify `axis=-1` to compute the L2_norm of each eigenmode individually.

In [6]:
modes_field.L2_norm(axis=-1)

array([1.00000043, 0.99983247, 0.99980958, 0.99975699, 0.99934917,
       0.99924161, 0.99973112, 0.99687801, 0.99607989, 0.99579105])

## Importing surfaces

The `Surface` class enable geometric computations on surfaces. We can construct surfaces from the COMSOL geometry by using the `surfaces_from_comsol_obj(mesh, comsol_obj)` function which extracts information about the sets of triangles which define the boundaries of the COMSOL mesh.

Since COMSOL objects often contain several surfaces it is necessary to determine the surface of interest by inspection. Here, we identify the last surface as the top surface of the device by looking at the number of triangles and the positions of points on the surface.

In [7]:
surfaces = surfaces_from_comsol_obj(mesh, cobj)
top_surface = surfaces[-1]
top_surface

Surface(
    mesh=Mesh(n_points=2823, n_tetrahedra=13851),
    n_triangles=582
)

In [8]:
# Plot points on the top surface of the device
show(plot_points(top_surface.points()))

## Random sampling

We can sample random points and field values on the surface of a device using the `random_point_sample(n_samples)` and `random_value_sample(field, n_samples)` methods of the `Surface` class.

In [9]:
# Sample random points on the top surface
points = top_surface.random_point_sample(n_samples=1000)
show(plot_points(points))

## Relative frequency shift due to physiosorption

The relative frequency shift in the $n$-th mode due to the physiosorption of an analyte on the surface of a device is given by,
$$ \frac{\Delta f_n}{f_n} = - \frac{m}{2 M} \| \phi_n(x) \|^2, $$
where $\Delta f_n / f_n$ is the relative frequency shift, $\phi_n(x)$ is the $n$-th eigenmode and $m$, $M$ are the masses of the analyte and device, respectively. This formula assumes that the eigenmodes have unity modal-mass.

We can now compute the frequency shifts due to analyte physiorption using the `random_value_sample(field, n_samples)` method of the `Surface` class.

In [10]:
# Compute the values of the eigenmodes at random points on the top surface
points, values = top_surface.random_value_sample(modes_field, n_samples=1000)

# Compute the relative frequency shifts by taking norm along the last axis
# of the values array
freq_shifts = -0.5 * np.linalg.norm(values, axis=-1) ** 2

In [11]:
# Plot frequency-shifts
mode_idxs = (4, 5)
mode_label = lambda i: f'$$\Delta f_{i} / f_{i}$$'

p = figure(
    title=f'Relative frequency shifts',
    x_axis_label=mode_label(mode_idxs[0] + 1),
    y_axis_label=mode_label(mode_idxs[1] + 1),
)
p.scatter(freq_shifts[:, 5], freq_shifts[:, 6])

p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'

p.title.text_font_size = '14pt'
p.toolbar.logo = None

show(p)